In [ ]:
import uproot
import awkward

Let's load the example data in [NanoAOD](https://twiki.cern.ch/twiki/bin/view/CMSPublic/WorkBookNanoAOD) format:

In [ ]:
# Create a TTree from root
tree = uproot.open("data/nano_dy.root")["Events"]
# TTree -> awkward.Array[awkward.Record[str, awkward.Array]]
array = tree.arrays(ak_add_doc=True)

In [ ]:
array.show()

The resulting data is a list of records. Each record represents a single event and all of its parameters data. For example here's some of the data for the first event in our file:

In [ ]:
array[0].show(50)

## Awkward-zipper example usage

The goal of the awkward-zipper package is to restructure the record of each event. These records are restructured in the same manner as in [Coffea package](https://coffea-hep.readthedocs.io/en/v2025.1.1/api/coffea.nanoevents.NanoAODSchema.html).

In [ ]:
from awkward_zipper import NanoAOD

restructure = NanoAOD(version="latest")
result = restructure(array)

In [ ]:
awkward.materialize(result)
result

Now let's go step by step how awkward-zipper reconstructs the original NanoAOD data.

## How the new fields are added


In [ ]:
from awkward_zipper.kernels import (
    counts2offsets,
    local2globalindex,
)

For example we have a local index branches with names matching `{source}_{target}Idx*` are converted to global indexes for the event chunk (postfix `G`).
(All local indices and their correlating global indices are taken from `NanoAOD.all_cross_references` dictionary)

In [ ]:
local_index = 'Jet_electronIdx1'

# cross_referense = NanoAOD.all_cross_references[local_index]
# global_index = "n" + cross_referense
global_index = 'nElectron'

array["Jet_electronIdx1G"] = local2globalindex(array[local_index], array[global_index])
array["Jet_electronIdx1G"]

How the functions like local2globalindex work is the main difference between awkward-zipper and coffea.

awkward-zipper does its inner calculations on awkward arrays, while coffea does them using [forms and buffers](https://awkward-array.org/doc/main/reference/generated/ak.to_buffers.html).

This change will make it easier for users to create their own ‘schemas’ (or modify existing ones)



These fields are then grouped by name.

Finally, all collections are then zipped into one NanoEvents record and returned.


## These fields are then grouped by name, where if:

one branch exists named name and no branches start with name_, it gets interpreted as a single flat array;

In [ ]:
# Example: Each event has only one Run Id. Interpreted flat array will look look like this:
result.run

one branch exists named name, one named n{name}, and no branches start with name_, it gets interpreted as a single jagged array;

In [ ]:
# Example: Each event has a flat array of PS Weights. Interpreted single jagged array will look look like this:
result.PSWeight

no branch exists named {name} and many branches start with name_*, they get interpreted as a flat table; or

In [ ]:
#Example: Each event has a SINGLE Generator. Each Generator consists of a record of Generator parameters. These parameters can be scalars or flat arrays. Interpreted flat table will look look like this:
result.Generator

one branch exists named n{name} and many branches start with name_*, they interpreted as a jagged table.

In [ ]:
# Example: Each event has an array of Jets. Each Jet consists of a record of Jet parameters. These parameters can be scalars or flat arrays. Interpreted jagged table will look look like this:
result.Jet

Finally, all collections are then zipped into one NanoEvents record and returned.

Final result:

In [ ]:
result

In [ ]:
result.Jet.mass

## Zipper with virtual arrays

Let's load the same root file but as virtual arrays. Virtual arrays don't load the data from disk (or in other words don't materialize the data).

In [ ]:
# Create a TTree from root
tree = uproot.open("data/nano_dy.root")["Events"]
# to load virtual arrays
access_log = [] # which of the data was materialized
# TTree -> awkward.Array[awkward.Record[str, awkward.Array]]
array = tree.arrays(ak_add_doc=True, access_log=access_log, virtual=True)

Calling zipper

In [ ]:
restructure = NanoAOD(version="latest")
result = restructure(array)

In [ ]:
access_log

In [ ]:
result

In [ ]:
result.Jet

In [ ]:
access_log

## Example calculation of a Z-peak

In [ ]:
restructure = NanoAOD(version="latest")
result = restructure(array)

In [ ]:
zcands = awkward.combinations(result.Muon, 2)

In [ ]:
access_log

In [ ]:
# calculate invariant mass
mass = awkward.flatten((zcands["0"] + zcands["1"]).mass)
mass

In [ ]:
access_log

We can see that for this, 4-vector coordinates were loaded, which were used to add the combinations